In [1]:
import warnings
import os
warnings.filterwarnings("ignore")

os.makedirs('./data',exist_ok=True)
os.makedirs('./utils',exist_ok=True)

#download custom processing scripts if not already downloaded
# !wget -P . https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/kinematics.py
# !wget -P . https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/circstat.py
# !wget -P . https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/processing.py
# !wget -P . https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/skeleton.py

#download example data or upload your own json annotations

#--------------------------------------------------------------------------------------
# Download example raw data from figshare or specify path to your own json annotations |
#--------------------------------------------------------------------------------------

#!wget -P . https://figshare.com/ndownloader/articles/25316500/versions/1
# #unzip data into ./data folder and remove zip file
# !unzip ./1 -d ./data
# !rm ./1

In [2]:
#uncomment install if running on google colab
#%pip install scikit-video

import sys, os, cv2, glob, json, gc
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
import json
import numpy as np
import pandas as pd
import itertools
from itertools import chain
from moviepy.editor import VideoFileClip
import skvideo.io
from tqdm import tqdm
import circstat as CS
import scipy as sc
import math

from processing import *
from kinematics import *
from skeleton import *

DEIDENTIFY = True #set to True to deidentify data, and exclude xy coordinates and pixel values from output
OVERWRITE = True
USE_CENTER_INSTANCE = False
USE_BEST_INSTANCE = True

dataset = 'gma_score_prediction'
json_path = f'./data/Infant Pose Data/{dataset}/annotations'
json_files = os.listdir(json_path)
directory = f'./data'

save_path = f'./pose_estimates/{dataset}_norm'

if not os.path.exists(save_path):
    os.makedirs(save_path)

kp_mapping = {0:'Nose', 1:'Neck', 2:'RShoulder', 3:'RElbow', 4:'RWrist', 5:'LShoulder', 6:'LElbow',
              7:'LWrist', 8:'RHip', 9:'RKnee', 10:'RAnkle', 11:'LHip',
              12:'LKnee', 13:'LAnkle', 14:'REye', 15:'LEye', 16:'REar', 17:'LEar'}

# Define the DataFrame columns as specified
columns = ['video_number', 'video', 'bp', 'frame', 'x', 'y', 'c','fps', 'pixel_x', 'pixel_y', 'time', 'part_idx']
data = []  # This will hold the data to be loaded into the DataFrame

vid_info = pd.read_csv(f'./data/{dataset}_video_info.csv')

In [ ]:
# format files as pkl with openpose standard and bodypart labels
for file_number, file in enumerate(tqdm(json_files)):
    # Construct the full file path
    file_path = os.path.join(json_path, file)
    fname = file.split('.')[0]
    interim = []

    if not OVERWRITE and os.path.exists(f'{save_path}/{fname}.pkl'):
        continue

    # Open and load the JSON data
    with open(file_path, 'r') as f:
        frames = json.load(f)
        info = vid_info[vid_info['video'] == fname]
        fps = vid_info['fps'].values[0]

        pixel_x = vid_info['width'].values[0]
        pixel_y = vid_info['height'].values[0]
        
        center_x = pixel_x / 2
        center_y = pixel_y / 2
        
        # Iterate through each frame in the JSON file
        for frame in frames:
            frame_id = frame['frame_id']
            if 'instances' in frame and len(frame['instances']) > 0:

                if USE_CENTER_INSTANCE:
                    instance_id = get_center_instance(frame['instances'], center_x, center_y)
                elif USE_BEST_INSTANCE:
                    instance_id = get_best_instance(frame['instances'])
                else:
                    instance_id = 0

                keypoints = frame['instances'][instance_id]['keypoints']
                confidence = frame['instances'][instance_id]['keypoint_scores']
                keypoints, confidence = convert_coco_to_openpose(keypoints, confidence)

                # Iterate through each keypoint
                for part_idx, (x, y) in enumerate(keypoints):

                    bp = kp_mapping[part_idx]
                    fps = fps
                    time = frame_id / fps
                    c = confidence[part_idx]

                    row = [file_number, fname, bp, frame_id, x, y, c, fps, pixel_x, pixel_y, time, part_idx]
                    interim.append(row)

    interim_df = pd.DataFrame(interim, columns=columns)
    interim_df.to_pickle(f'{save_path}/{fname}.pkl')

    del interim_df


In [ ]:
# Ensure the save_path directory exists
save_path = f'./pose_estimates/{dataset}_norm'

if os.path.exists(f'{save_path}/pose_estimates_{dataset}.csv'):
    os.remove(f'{save_path}/pose_estimates_{dataset}.csv')
    print('Removed existing CSV file')

for pklfile in tqdm(os.listdir(save_path)):
    if not pklfile.endswith('.pkl'):
        continue
    else:
        interim_df = pd.read_pickle(f'{save_path}/{pklfile}')
        interim_df.to_csv(f'{save_path}/pose_estimates_{dataset}.csv', mode='a', header=False, index=False)

    del interim_df

In [ ]:
csv_path = f'{save_path}/pose_estimates_{dataset}.csv'
output_csv_path = f'{save_path}/pose_estimates_{dataset}_b.csv'
chunksize = 1000  # Number of rows per chunk

# Define the new headers
new_headers = ['video_number', 'video', 'bp', 'frame', 'x', 'y', 'c', 'fps', 'pixel_x', 'pixel_y', 'time', 'part_idx']

# Read the CSV file in chunks
chunk_iterator = pd.read_csv(csv_path, chunksize=chunksize)

# Process the first chunk
first_chunk = next(chunk_iterator)
first_chunk.columns = new_headers
first_chunk.to_csv(output_csv_path, mode='w', index=False)

# Process the rest of the chunks and append them to the new CSV file without headers
for chunk in chunk_iterator:
    chunk.columns = new_headers
    chunk.to_csv(output_csv_path, mode='a', index=False, header=False)

# rename the csv file
os.rename(csv_path, f'{save_path}/pose_estimates_{dataset}_x.csv')
os.rename(output_csv_path, csv_path)


In [ ]:
def process_dataframe(df, pose_estimate_path):
    if df.empty:
        #print("DataFrame is empty, skipping processing.")
        return
    print(f"Processing DataFrame for video_number: {df['video_number'].iloc[0]}")
    try:
        if dataset == 'Youtube':
            session = df['video'].unique()[0].split('_')[1][0]
            infant = df['video'].unique()[0].split('_')[1][3:]
        elif dataset == 'Clinical':
            # split based on naming convention
            session = df['video'].unique()[0].split('_')[1][1]
            infant = df['video'].unique()[0].split('_')[0][-1]
        elif dataset == 'gma_score_prediction': 
            session = 0
            infant = df['video'].unique()[0]

        age = '3_4 Month'

        print(f'infant: {infant}')
        
    except:
        f'could not process video {df["video"].unique()[0]}'
        return

    median_window = 1
    mean_window = 1
    delta_window = 0.25  # Smoothing applied to delta_x, velocity, acceleration

    df['x'] = pd.to_numeric(df['x'])
    df['y'] = pd.to_numeric(df['y'])

    # Interpolate
    df = df.groupby(['video', 'bp']).apply(interpolate_df).reset_index(drop=True)

    # Median and mean filter
    median_window = 0.5
    mean_window = 0.5
    df = df.groupby(['video', 'bp']).apply(lambda x: smooth(x, 'y', median_window, mean_window)).reset_index(drop=True)
    df = df.groupby(['video', 'bp']).apply(lambda x: smooth(x, 'x', median_window, mean_window)).reset_index(drop=True)

    try:
        # Rotate and normalise by reference
        xdf = normalise_skeletons(df)
        xdf = get_dynamics_xy(xdf, delta_window)
            
        xdf.to_pickle(f'{pose_estimate_path}/xdf/{infant}_{session}_processed_pose_estimates_coords_norm.pkl')

        adf = get_joint_angles(df)
        adf = get_dynamics_angle(adf, delta_window)
        adf.to_pickle(f'{pose_estimate_path}/adf/{infant}_{session}_processed_pose_estimates_coords_norm.pkl')

    except KeyError as e:
        print(f'Error processing video for {infant}: {e}')


In [3]:
# Smooth detections and compute features
pose_estimate_path = f'./pose_estimates/{dataset}_norm'
csv_path = f'{pose_estimate_path}/pose_estimates_{dataset}.csv'
save_path = f'{pose_estimate_path}/pose_estimates_{dataset}_processed.csv'

# List of subdirectories to create
subdirs = [
    "",
    "xdf",
    "adf",
    "xy_features",
    "angle_features",
    "xy_features/total",
    "angle_features/total",
    "xy_features/windows",
    "angle_features/windows"
]

# Create necessary directories
for subdir in subdirs:
    os.makedirs(f'{pose_estimate_path}/{subdir}', exist_ok=True)


In [ ]:
from IPython.display import clear_output
chunksize = 100000
buffer = pd.DataFrame()

# Read the CSV file in chunks
chunk_iterator = pd.read_csv(csv_path, chunksize=chunksize)

for chunk in chunk_iterator:
  unique_videos = chunk['video_number'].unique()

  for video_number in unique_videos:
      video_chunk = chunk[chunk['video_number'] == video_number]

      if not buffer.empty:
        if buffer['video_number'].iloc[0] == video_number:

            buffer = pd.concat([buffer, video_chunk], ignore_index=True)
            if video_number not in chunk['video_number'].values:
                process_dataframe(buffer, pose_estimate_path)
                buffer = pd.DataFrame()

        else:
            process_dataframe(buffer, pose_estimate_path)
            buffer = video_chunk
      else:
          buffer = video_chunk
  clear_output(wait=True)
  chunk = chunk[~chunk['video_number'].isin(unique_videos)]

# # Process any remaining rows in the buffer
if not buffer.empty:
    print("Processing remaining rows in the buffer...")
    process_dataframe(buffer, pose_estimate_path)

In [ ]:
# Compute angular features

HEADER_WRITTEN = False

def process_adf_file(file): 
        if 'pkl' in file:
             adf = pd.read_pickle(os.path.join(f'{pose_estimate_path}/adf', file))
        else:
            return       

        video_number = adf.video.unique()[0]

        # Compute window angle features: 
        mean_type = 'windows'
        window_size = 2*int(adf['fps'].iloc[0]) # 2 seconds

        # Compute window angle features: 
        feature_angle = adf.groupby(['bp','video']).apply(rolling_angle_features, window_size=window_size).reset_index(drop=True)
        feature_angle = pd.pivot_table(feature_angle, index=['video','frame'], columns=['bp'])
        l0 = feature_angle.columns.get_level_values(1)
        l1 = feature_angle.columns.get_level_values(0)
        cols = [l1[i]+'_'+l0[i] for i in range(len(l1))]
        feature_angle.columns = cols
        feature_angle =feature_angle.reset_index()

        # - measure of symmetry (left-right cross correlation)
        corr_joint = adf.groupby(['video', 'part']).apply(rolling_lrcorr_angle_features, window_size=window_size, min_periods=1, var='angle')
        corr_joint.reset_index(inplace=True)
        corr_joint.drop(columns=['level_2','R','L'],inplace=True)
        corr_joint['part'] = 'lrCorr_angle_'+corr_joint['part']
        corr_joint.columns = ['video', 'feature', 'frame','Value']
        corr_joint = pd.pivot_table(corr_joint, index=['video','frame'], columns=['feature'])
        l1 = corr_joint.columns.get_level_values(1)
        corr_joint.columns = l1
        corr_joint = corr_joint.reset_index()

        feature_angle = pd.merge(feature_angle,corr_joint, on=['video','frame'], how='outer')
        feature_angle.to_csv(f'{pose_estimate_path}/angle_features/{mean_type}/{video_number}_features_{mean_type}_angle.csv', header=True, index=False)

        #compute total angle features (average by video)
        mean_type = 'total'
        feature_angle = adf.groupby(['bp','video']).apply(angle_features).reset_index(drop=True)
        feature_angle = pd.pivot_table(feature_angle, index='video', columns=['bp'])        
        
        l0 = feature_angle.columns.get_level_values(1)
        l1 = feature_angle.columns.get_level_values(0)
        cols = [l1[i]+'_'+l0[i] for i in range(len(l1))]
        feature_angle.columns = cols
        feature_angle =feature_angle.reset_index()

        # - measure of symmetry (left-right cross correlation)
        corr_joint = adf.groupby(['video', 'part']).apply(lambda x:corr_lr(x,'angle')).reset_index()
        corr_joint['part'] = 'lrCorr_angle_'+corr_joint['part']
        corr_joint.columns = ['video', 'feature', 'Value']
        corr_joint = pd.pivot_table(corr_joint, index='video', columns=['feature'])
        l1 = corr_joint.columns.get_level_values(1)
        corr_joint.columns = l1
        corr_joint = corr_joint.reset_index()
        feature_angle = pd.merge(feature_angle,corr_joint, on='video', how='outer')

        feature_angle.to_csv(f'{pose_estimate_path}/angle_features/{mean_type}/{video_number}_features_{mean_type}_angle.csv', header=True, index=False)
        
        return

In [ ]:
# Compute XY features (whole video average)
HEADER_WRITTEN = False

def process_xdf_file(file): 
        if 'pkl' in file:
             xdf = pd.read_pickle(os.path.join(f'{pose_estimate_path}/xdf', file))
        else:
            return
        
        bps = ['LAnkle', 'RAnkle', 'LWrist', 'RWrist']
        filtered_xdf = xdf[np.isin(xdf.bp, bps)]
        video_number = xdf.video.unique()[0]
        
        # Compute window xy features: 

        feature_total_xy = filtered_xdf.groupby(['bp','video']).apply(xy_features).reset_index(drop=True)
        feature_window_xy = filtered_xdf.groupby(['bp', 'video']).apply(lambda group: compute_rolling_features(group, sampling_rate=int(group['fps'].iloc[0]))).reset_index(drop=True)
        
        for feature_xy in [feature_total_xy, feature_window_xy]:
            
            if feature_xy.columns.str.contains('window_number').any():
                feature_xy = pd.pivot_table(feature_xy, index=['video','window_number','t_start'], columns=['bp'])
                mean_type = 'windows'
            else:
                feature_xy = pd.pivot_table(feature_xy, index='video', columns=['bp'])
                mean_type = 'total'
            
            l0 = feature_xy.columns.get_level_values(1)
            l1 = feature_xy.columns.get_level_values(0)
            cols = [l1[i]+'_'+l0[i] for i in range(len(l1))]
            feature_xy.columns = cols
            feature_xy = feature_xy.reset_index()
            # - measure of symmetry (left-right cross correlation)
            xdf['dist'] = np.sqrt(xdf['x']**2+xdf['y']**2)
            corr_joint = xdf.groupby(['video', 'part']).apply(lambda x:corr_lr(x,'dist')).reset_index()
            corr_joint['part'] = 'lrCorr_x_'+corr_joint['part']
            corr_joint.columns = ['video', 'feature', 'Value']
            corr_joint = pd.pivot_table(corr_joint, index='video', columns=['feature'])
            l1 = corr_joint.columns.get_level_values(1)
            corr_joint.columns = l1
            corr_joint = corr_joint.reset_index()
            feature_xy = pd.merge(feature_xy, corr_joint, on='video', how='outer')

            feature_xy.to_csv(f'{pose_estimate_path}/xy_features/{mean_type}/{video_number}_features_{mean_type}_xy.csv', header=True, index=False)
        
        return



In [132]:
file = '/workspaces/gma_score_prediction_from_video/pose_estimates/gma_score_prediction_norm/adf/599_0_processed_pose_estimates_coords_norm.pkl'
#process_adf_file(file)
adf = pd.read_pickle(file)
video_number = adf.video.unique()[0]
window_size = 2*int(adf['fps'].iloc[0]) # 2 seconds

# Compute window angle features: 
mean_type = 'windows'
feature_angle = adf.groupby(['bp','video']).apply(rolling_angle_features, window_size=window_size).reset_index(drop=True)
feature_angle = pd.pivot_table(feature_angle, index=['video','frame'], columns=['bp'])
l0 = feature_angle.columns.get_level_values(1)
l1 = feature_angle.columns.get_level_values(0)
cols = [l1[i]+'_'+l0[i] for i in range(len(l1))]
feature_angle.columns = cols
feature_angle =feature_angle.reset_index()

# - measure of symmetry (left-right cross correlation)
corr_joint = adf.groupby(['video', 'part']).apply(rolling_lrcorr_angle_features, window_size=window_size, min_periods=1, var='angle')
corr_joint.reset_index(inplace=True)
corr_joint.drop(columns=['level_2','R','L'],inplace=True)
corr_joint['part'] = 'lrCorr_angle_'+corr_joint['part']
corr_joint.columns = ['video', 'feature', 'frame','Value']
corr_joint = pd.pivot_table(corr_joint, index=['video','frame'], columns=['feature'])
l1 = corr_joint.columns.get_level_values(1)
corr_joint.columns = l1
corr_joint = corr_joint.reset_index()

feature_angle = pd.merge(feature_angle,corr_joint, on=['video','frame'], how='outer')


In [ ]:
feature_angle = pd.merge(feature_angle,corr_joint, on='window_number', how='outer')
feature_angle.video = int(feature_angle.video.unique()[0])

feature_angle.to_csv(f'{pose_estimate_path}/angle_features/{mean_type}/{video_number}_features_{mean_type}_angle.csv', header=True, index=False)


In [ ]:


feature_angle = adf.groupby(['bp','video']).apply(rolling_angle_features, window_size=int(adf['fps'].iloc[0]), min_periods=1)

feature_angle.head()

frame  video      bp  mean_angle  stdev_angle  entropy_angle  \
bp     video                                                                   
LElbow 599   0     14    599  LElbow   94.155699     0.000000       0.000000   
             1     15    599  LElbow   91.105756     2.156381       0.693147   
             2     16    599  LElbow   88.139582     3.447785       1.098612   
             3     17    599  LElbow   85.578878     4.326273       1.386294   
             4     18    599  LElbow   83.456862     4.891072       1.609438   

                median_vel  IQR_vel  IQR_acc  
bp     video                                  
LElbow 599   0         NaN      NaN      NaN  
             1         NaN      NaN      NaN  
             2         NaN      NaN      NaN  
             3         NaN      NaN      NaN  
             4         NaN      NaN      NaN

In [ ]:



corr_joint.head()

frame           R          L  rolling_corr
video part                                               
599   Elbow 0     14  153.730411  94.155699           NaN
            1     15  152.520951  88.055813      1.000000
            2     16  151.314750  82.207692      0.999935
            3     17  150.019914  77.904200      0.995570
            4     18  148.587739  74.988324      0.984529

In [75]:
corr_joint.reset_index(drop=True).rolling(window=30, min_periods=1).apply(lambda x: x['R'].corr(x['L']), raw=False)

KeyError: 'R'

In [80]:
corr_joint.reset_index()

,video,part,level_2,R,L
0,599,Elbow,0,153.730411,94.155699
1,599,Elbow,1,152.520951,88.055813
2,599,Elbow,2,151.314750,82.207692
3,599,Elbow,3,150.019914,77.904200
4,599,Elbow,4,148.587739,74.988324
...,...,...,...,...,...
11763,599,Shoulder,2937,203.872332,189.461361
11764,599,Shoulder,2938,203.390789,185.202000
11765,599,Shoulder,2939,202.458171,181.221585
11766,599,Shoulder,2940,201.220531,177.955516


In [32]:
corr_joint.window_number.max()

100

In [31]:
feature_angle = pd.pivot_table(feature_angle, index=['video','window_number','t_start'], columns=['bp'])
feature_angle

IQR_acc_angle                         \
bp                                   LElbow       LHip       LKnee   
video window_number t_start                                          
599   13            0.467133     688.002832  72.254536  231.912567   
      14            0.467133     683.495490  73.245011  237.021667   
      15            0.500500     678.988148  76.140182  239.984430   
      16            0.533867     674.480805  79.460017  242.015670   
      17            0.567233     669.973463  81.724484  244.330205   
...                                     ...        ...         ...   
      2880          96.096000    766.583518  30.815281   46.209524   
      2881          96.129367    730.702761  33.458833   49.671109   
      2882          96.162733    730.702761  35.805122   52.843332   
      2883          96.196100    730.702761  38.112579   54.557208   
      2884          96.229467    730.702761  40.212800   57.145320   

                                                                              \
bp                              LShoulder      RElbow       RHip       RKnee   
video window_number t_start                                                    
599   13            0.467133   172.849867  132.521153  52.633768  386.239566   
      14            0.467133   170.002278  134.878925  52.066604  383.060888   
      15            0.500500   171.018753  141.698536  51.499439  379.882209   
      16            0.533867   172.281041  150.354511  50.932275  376.703531   
      17            0.567233   170.170890  158.221374  51.769783  373.524852   
...                                   ...         ...        ...         ...   
      2880          96.096000  882.048330   83.479679  40.873270   43.164033   
      2881          96.129367  882.048330   83.479679  40.182983   41.724360   
      2882          96.162733  882.048330   76.646340  40.182983   40.118061   
      2883          96.196100  882.048330   64.693294  40.182983   36.968443   
      2884          96.229467  882.048330   50.162649  40.182983   36.396216   

                                         IQR_vel_angle             ...  \
bp                             RShoulder        LElbow       LHip  ...   
video window_number t_start                                        ...   
599   13            0.467133   77.068315     86.050707  15.075224  ...   
      14            0.467133   78.205320     82.506671  15.390043  ...   
      15            0.500500   77.408739     80.740461  15.256227  ...   
      16            0.533867   76.612158     78.974250  15.268586  ...   
      17            0.567233   75.815576     77.208039  15.274336  ...   
...                                  ...           ...        ...  ...   
      2880          96.096000  79.721294    116.071001   8.107089  ...   
      2881          96.129367  89.310347    116.071001   8.107089  ...   
      2882          96.162733  92.735169    117.431225   8.107089  ...   
      2883          96.196100  92.735169    120.757808   8.107089  ...   
      2884          96.229467  92.735169    123.535458   8.107089  ...   

                              median_vel_angle            stdev_angle  \
bp                                       RKnee  RShoulder      LElbow   
video window_number t_start                                             
599   13            0.467133         32.466032  14.691694    2.945662   
      14            0.467133         31.405292  14.699892    2.932862   
      15            0.500500         31.001130  14.892209    2.869519   
      16            0.533867         30.596968  15.084525    2.783276   
      17            0.567233         29.966497  15.286897    2.671249   
...                                        ...        ...         ...   
      2880          96.096000         9.385993  24.624851    4.937362   
      2881          96.129367         8.380726  24.624851    4.837663   
      2882          96.162733         7.123569  24.624851    4.731620   
      2883     

In [ ]:
import multiprocessing

pool = multiprocessing.Pool(processes=20)  # Adjust the number of processes as needed
file_paths = os.listdir(f'{pose_estimate_path}/xdf')  # List of file paths
results = pool.map(process_xdf_file, file_paths)


In [5]:
import multiprocessing

pool = multiprocessing.Pool(processes=20)  # Adjust the number of processes as needed
file_paths = os.listdir(f'{pose_estimate_path}/adf')  # List of file paths
results = pool.map(process_adf_file, file_paths)

KeyboardInterrupt: 

In [ ]:
# Combine features
SAVE = True

features_xy = pd.read_csv(f'{pose_estimate_path}/features_xy.csv')
features_angle = pd.read_csv(f'{pose_estimate_path}/features_angle.csv')

features = pd.merge(features_xy, features_angle, on='video', how='inner')

In [ ]:
# Split features based on video file naming convention

if dataset == 'Youtube':
    features['infant'] = features['video'].str.split('_').str.get(1).str[-3:]
    features['session'] = features['video'].str.split('_').str.get(1).str[0]
    features['age'] = 'month'

elif dataset == 'Clinical':
    features['infant'] = features['video'].str.split('_').str.get(0).str[-1]
    features['session'] = features['video'].str.split('_').str.get(1).str[1]
    features['age'] = 'month'

elif dataset == 'gma_score_prediction':
    features['infant'] = features['video']
    features['session'] = 0
    features['age'] = 'month'

if SAVE:
    features.to_csv(f'{pose_estimate_path}/features.csv', header=True, index=False)

In [ ]:
# Body parts and sides
body_parts = ["Knee", "Shoulder", "Hip", "Elbow", "Wrist", "Ankle"]
sides = ["L", "R"]

# Function to split the feature string into "feature", "part", and "side"
def split_feature(feature):
    # Split the feature into components
    parts = feature.split('_')
    feature_name = "_".join(parts[:-1])  # Default to everything before the last part
    part = parts[-1] if len(parts) > 1 else ""  # Default to the last part

    side = ""

    # Check if the last part has a side
    for body_part in body_parts:
        if body_part in part:  # Find body part
            idx = part.index(body_part)
            if idx > 0 and part[idx-1] in sides:  # Check for side prefix
                side = part[idx-1]
                part = part[idx:]  # Remove the side from the part
            break  # Exit loop after finding the body part

    # Adjust feature name to remove body part and side, if found
    if part in feature_name:
        feature_name = feature_name.replace(part, "").strip("_")
    if side in feature_name:
        feature_name = feature_name.replace(side, "").strip("_")

    return pd.Series([feature_name, part, side])

In [ ]:
# Include data for the following, and restructure dataframe
# Customize for your specific dataset
id_vars = ['infant', 'age','session','video']
melted = pd.melt(features, id_vars=id_vars, var_name="feature", value_name="Value")

# Apply the function to the 'feature' column in your melted DataFrame
melted[['feature', 'part', 'side']] = melted['feature'].apply(split_feature)
melted = melted.dropna()


In [ ]:
melted.head()

In [ ]:
mean = melted.groupby(['infant', 'age', 'session', 'video', 'feature', 'part'])['Value'].mean().reset_index()
mean.to_csv(f'{pose_estimate_path}/{dataset}_features_mean_by_side.csv', header=True, index=False)

mean.head()